# TP 10 – Dates, day count et chargement de données Yahoo Finance

Ce TP couvre les modules `date` et `market.loader`.

Objectifs :
- manipuler la classe `Date` et un day counter simple;
- comprendre comment `YahooFinanceLoader` construit un `MarketSurface` à partir de données de marché réelles;
- esquisser un workflow complet : données brutes → surface de prix → surface de volatilité implicite.

In [ ]:
from fypy.date.Date import Date
from fypy.date.DayCounter import DayCounter_365
from fypy.market.loader.YahooFinanceSurfaceLoader import YahooFinanceLoader
from fypy.termstructures.DiscountCurve import DiscountCurve_ConstRate
from fypy.termstructures.EquityForward import EquityForward
from fypy.volatility.implied.ImpliedVolCalculator import ImpliedVolCalculator_Black76

%matplotlib inline

## 1. Classe Date et DayCounter

`Date` étend `datetime` avec quelques utilitaires :
- construction à partir de string (`from_str`), de `datetime` ou `datetime.date`;
- addition/soustraction de jours via opérateurs `+` et `-`;
- fonction statique `days_between`.

`DayCounter` fournit une interface pour convertir un nombre de jours en fraction d’année. Par exemple,
`DayCounter_365` implémente la convention **ACT/365**.


In [ ]:
# Exemple simple de manipulations de dates et day count
d0 = Date.from_str('2023-01-10')
d1 = Date.from_str('2023-07-25')

dc = DayCounter_365()
nb_jours = Date.days_between(start=d0, end=d1)
year_frac = dc.year_fraction(start=d0, end=d1)

print('Date de départ  :', d0.date())
print('Date d\'arrivée :', d1.date())
print('Nombre de jours :', nb_jours)
print('Year fraction (ACT/365) :', year_frac)

Ces objets sont utilisés par le loader Yahoo pour convertir des maturités calendaires (dates d'expiration) en
temps de maturité \( T \) continus pour les modèles (en années).

## 2. YahooFinanceLoader : de l’option chain à MarketSurface

`YahooFinanceLoader` fournit des méthodes pour :
- télécharger un "option chain" depuis Yahoo Finance (`load_df_from_api`);
- charger ces données depuis un CSV (`load_from_file`);
- convertir un `DataFrame` de quotes en `MarketSurface` (`load_from_frame`).

Chaque ligne de données contient typiquement :
- `expiry` : date d’expiration (string) ;
- `strike`, `isCall` ;
- `bid`, `ask`, `spot`, etc.

Le loader :
- utilise un `DayCounter` pour convertir `expiry` en TTM (time-to-maturity);
- crée pour chaque maturité un `MarketSlice` (avec `bid`, `ask`, `mid`);
- assemble ces slices dans un `MarketSurface` cohérent, avec courbes `discount` et `forward`.

In [ ]:
# ATTENTION : cette cellule nécessite l'accès réseau et l'API Yahoo Finance via yfinance.
# Elle peut échouer selon l'environnement d'exécution.

ticker = 'SPY'  # ETF S&P 500, très liquide
loader = YahooFinanceLoader()

try:
    # Récupération des quotes d'options depuis Yahoo Finance
    df = loader.load_df_from_api(ticker=ticker, volume_filter=0)
    print('Nombre de lignes téléchargées :', len(df))
except Exception as e:
    print('Erreur pendant le chargement Yahoo (environnement sans réseau ?) :', e)
    df = None

Si les données ont été téléchargées correctement, on peut construire une `MarketSurface` à partir du DataFrame.

In [ ]:
if df is not None:
    disc_curve = DiscountCurve_ConstRate(rate=0.02)
    div_disc = DiscountCurve_ConstRate(rate=0.0)

    surface = loader.load_from_frame(df=df,
                                     disc_curve=disc_curve,
                                     div_disc=div_disc,
                                     fit_discount=False)

    print('Surface construite avec', surface.num_slices, 'maturités distinctes.')
else:
    print('Pas de surface construite (df == None).')

## 3. De la surface de prix à la surface de volatilité implicite

Une fois la surface de prix disponible, il est naturel de calculer les **volatilités implicites** :
- on construit un `ImpliedVolCalculator_Black76` à partir de la courbe de forward et de la courbe d’actualisation;
- on appelle `surface.fill_implied_vols(calculator)` pour propager les vols dans chaque slice;
- on peut ensuite utiliser `ModelVolSurfaceSlices` (voir TP 8) pour interpoler/extrapoler la surface.


In [ ]:
if df is not None:
    # Reconstituer les courbes à partir de la surface
    fwd_curve = surface.forward_curve
    disc_curve_surf = surface.discount_curve

    ivc = ImpliedVolCalculator_Black76(disc_curve=disc_curve_surf, fwd_curve=fwd_curve)
    surface.fill_implied_vols(calculator=ivc)

    # Affichons quelques stats simples sur les vols
    for T, sl in surface.slices.items():
        if sl.mid_vols is not None:
            print(f'T = {T:.2f} ans : vol imp. moyenne = {sl.mid_vols.mean():.4f}')
else:
    print('Pas de vols implicites calculés (pas de surface).')

Ce TP montre comment :
- passer de dates réelles à des maturités continues \( T \) via un day counter;
- utiliser `YahooFinanceLoader` pour construire un `MarketSurface` à partir de données réelles;
- transformer cette surface de prix en surface de volatilité implicite, prête à être utilisée pour calibration.

**Pour aller plus loin :**
- sauvegarder le DataFrame dans un CSV puis le recharger avec `load_from_file`;
- filtrer la surface (strikes, maturités) avant d’entreprendre une calibration (TP 9).